# Semantinis branduolys

Šiame kodo pavyzdyje naudosite [Semantic Kernel](https://aka.ms/ai-agents-beginners/semantic-kernel) AI sistemą, kad sukurtumėte pagrindinį agentą.

Šio pavyzdžio tikslas yra parodyti jums veiksmus, kuriuos vėliau naudosime kituose kodo pavyzdžiuose, įgyvendindami skirtingus agentinius modelius.


## Importuokite reikalingas Python paketus


In [ ]:
import os 
from typing import Annotated
from openai import AsyncOpenAI

from dotenv import load_dotenv

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function

## Kliento kūrimas

Šiame pavyzdyje naudosime [GitHub Models](https://aka.ms/ai-agents-beginners/github-models), kad galėtume pasiekti LLM.

`ai_model_id` yra apibrėžtas kaip `gpt-4o-mini`. Pabandykite pakeisti modelį į kitą, kuris yra prieinamas GitHub Models rinkoje, kad pamatytumėte skirtingus rezultatus.

Norėdami naudoti `Azure Inference SDK`, kuris naudojamas `base_url` GitHub Models, pasitelksime `OpenAIChatCompletion` jungtį Semantic Kernel aplinkoje. Taip pat yra ir kitų [galimų jungčių](https://learn.microsoft.com/semantic-kernel/concepts/ai-services/chat-completion), leidžiančių naudoti Semantic Kernel su kitais modelių tiekėjais.


In [ ]:
import random   

# Define a sample plugin for the sample

class DestinationsPlugin:
    """A List of Random Destinations for a vacation."""

    def __init__(self):
        # List of vacation destinations
        self.destinations = [
            "Barcelona, Spain",
            "Paris, France",
            "Berlin, Germany",
            "Tokyo, Japan",
            "Sydney, Australia",
            "New York, USA",
            "Cairo, Egypt",
            "Cape Town, South Africa",
            "Rio de Janeiro, Brazil",
            "Bali, Indonesia"
        ]
        # Track last destination to avoid repeats
        self.last_destination = None

    @kernel_function(description="Provides a random vacation destination.")
    def get_random_destination(self) -> Annotated[str, "Returns a random vacation destination."]:
        # Get available destinations (excluding last one if possible)
        available_destinations = self.destinations.copy()
        if self.last_destination and len(available_destinations) > 1:
            available_destinations.remove(self.last_destination)

        # Select a random destination
        destination = random.choice(available_destinations)

        # Update the last destination
        self.last_destination = destination

        return destination

In [ ]:
load_dotenv()
client = AsyncOpenAI(
    api_key=os.environ.get("GITHUB_TOKEN"), 
    base_url="https://models.inference.ai.azure.com/",
)

# Create an AI Service that will be used by the `ChatCompletionAgent`
chat_completion_service = OpenAIChatCompletion(
    ai_model_id="gpt-4o-mini",
    async_client=client,
)

## Sukurti agentą

Toliau sukuriame agentą, pavadintą `TravelAgent`.

Šiame pavyzdyje naudojame labai paprastas instrukcijas. Galite pakeisti šias instrukcijas, kad pamatytumėte, kaip agentas reaguoja skirtingai.


In [ ]:
agent = ChatCompletionAgent(
    service=chat_completion_service, 
    plugins=[DestinationsPlugin()],
    name="TravelAgent",
    instructions="You are a helpful AI Agent that can help plan vacations for customers at random destinations",
)

## Agento paleidimas

Dabar galime paleisti agentą, apibrėždami giją, kurios tipas yra `ChatHistoryAgentThread`. Bet kokie būtini sistemos pranešimai pateikiami agento invoke_stream `messages` raktažodžio argumente.

Kai tai bus apibrėžta, sukuriame `user_inputs`, kuris bus tai, ką vartotojas siunčia agentui. Šiuo atveju nustatėme šį pranešimą kaip `Suplanuok man saulėtą atostogą`.

Drąsiai pakeiskite šį pranešimą, kad pamatytumėte, kaip agentas reaguoja skirtingai.


In [ ]:
async def main():
    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: ChatHistoryAgentThread | None = None

    user_inputs = [
        "Plan me a day trip.",
    ]

    for user_input in user_inputs:
        print(f"# User: {user_input}\n")
        first_chunk = True
        async for response in agent.invoke_stream(
            messages=user_input, thread=thread,
        ):
            # 5. Print the response
            if first_chunk:
                print(f"# {response.name}: ", end="", flush=True)
                first_chunk = False
            print(f"{response}", end="", flush=True)
            thread = response.thread
        print()

    # Clean up the thread
    await thread.delete() if thread else None

await main()


---

**Atsakomybės apribojimas**:  
Šis dokumentas buvo išverstas naudojant AI vertimo paslaugą [Co-op Translator](https://github.com/Azure/co-op-translator). Nors siekiame tikslumo, prašome atkreipti dėmesį, kad automatiniai vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojama profesionali žmogaus vertimo paslauga. Mes neprisiimame atsakomybės už nesusipratimus ar klaidingus interpretavimus, atsiradusius naudojant šį vertimą.
